In [ ]:
from beamtest_analysis_helper import plot_resolution_table, plot_resolution_with_pulls
import pandas as pd
import yaml

## Figure configs

In [ ]:
# with open("./board_configs_yaml/DESY_TB_2025Mar_Main.yaml", "r") as file:
#     fig_configs = yaml.safe_load(file)

# print(fig_configs.keys())

# given_run = 'run47'
# selected_fig_config = fig_configs[given_run]

# for id in [0, 1, 2, 3]:
#     selected_fig_config[id]['title'] = f"{selected_fig_config[id]['name']} HV{selected_fig_config[id]['HV']}V OS:{selected_fig_config[id]['offset']}"

# board_to_draw = [0, 1, 2]

In [ ]:
# board_to_analyze = [0,1,3]

with open("./board_configs_yaml/DESY_TB_2025Mar_Cooling.yaml", "r") as file:
    fig_configs = yaml.safe_load(file)

print(fig_configs.keys())

given_run = 'run47'
selected_fig_config = fig_configs[given_run]

for id in [0, 1, 2, 3]:
    selected_fig_config[id]['title'] = f"{selected_fig_config[id]['name']} HV{selected_fig_config[id]['HV']}V OS:{selected_fig_config[id]['offset']}"

board_to_draw = [0,1,3]

In [ ]:
## final_df: time resolution dataframe
# resolution_df = pd.read_csv(f'./desy_Mar2025_results/main_tele/{given_run}/resolution_Trig_Bar12_PT-NH2_HV{selected_fig_config[2]["HV"]}_IME-5_HV{selected_fig_config[3]["HV"]}.csv')
# resolution_df = pd.read_csv(f'./desy_Mar2025_results/main_tele/{given_run}/resolution_Trig_Bar12_PT-NH8_HV{selected_fig_config[2]["HV"]}_IME-5_HV{selected_fig_config[3]["HV"]}.csv')


# resolution_df = pd.read_csv(f'./desy_Mar2025_results/cooling_tele/{given_run}/resolution_IME-2_Neg15C_HV{selected_fig_config[0]["HV"]}.csv')
# resolution_df = pd.read_csv(f'./desy_Mar2025_results/cooling_tele/{given_run}/resolution_IME-2_Pos18C_HV{selected_fig_config[0]["HV"]}.csv')
# resolution_df = pd.read_csv(f'./desy_Mar2025_results/cooling_tele/{given_run}/resolution_IME-2_Pos23C_HV{selected_fig_config[0]["HV"]}.csv')
resolution_df = pd.read_csv(f'./desy_Mar2025_results/cooling_tele/{given_run}/resolution_PT-NH2_Neg24p5C_HV{selected_fig_config[0]["HV"]}.csv')


## nevt_track_df: number of events dataframe
# nevt_track_df = pd.read_csv(f'./desy_Mar2025_results/main_tele/{given_run}/Trig_Bar12_PT-NH2_HV{selected_fig_config[2]["HV"]}_IME-5_HV{selected_fig_config[3]["HV"]}_nevt_per_track.csv')
# nevt_track_df = pd.read_csv(f'./desy_Mar2025_results/main_tele/{given_run}/Trig_Bar12_PT-NH8_HV{selected_fig_config[2]["HV"]}_IME-5_HV{selected_fig_config[3]["HV"]}_nevt_per_track.csv')


# nevt_track_df = pd.read_csv(f'./desy_Mar2025_results/cooling_tele/{given_run}/IME-2_Neg15C_HV{selected_fig_config[0]["HV"]}_nevt_per_track.csv')
# nevt_track_df = pd.read_csv(f'./desy_Mar2025_results/cooling_tele/{given_run}/IME-2_Pos18C_HV{selected_fig_config[0]["HV"]}_nevt_per_track.csv')
# nevt_track_df = pd.read_csv(f'./desy_Mar2025_results/cooling_tele/{given_run}/IME-2_Pos23C_HV{selected_fig_config[0]["HV"]}_nevt_per_track.csv')
nevt_track_df = pd.read_csv(f'./desy_Mar2025_results/cooling_tele/{given_run}/PT-NH2_Neg24p5C_HV{selected_fig_config[0]["HV"]}_nevt_per_track.csv')

In [ ]:
merged_df = pd.merge(resolution_df, nevt_track_df, on=['row0', 'col0', 'row1', 'col1', 'row2', 'col2', 'row3', 'col3'])
# merged_df = pd.merge(resolution_df, nevt_track_df, on=['row0', 'col0', 'row1', 'col1', 'row3', 'col3'])

merged_df.sort_values(by=['nevt'], ascending=False).reset_index(drop=True)

In [ ]:
# merged_df.sort_values(by=['nevt'], ascending=False).reset_index(drop=True).head(10)

## Define condition by track pixel combinations from merged_df

In [ ]:
### 2025 Mar DESY TB - main telescope upto Run45
# condition = (merged_df['row1'] == merged_df['row0']) & \
#             (merged_df['col1'] == merged_df['col0']) & \
#             (merged_df['row1'] == merged_df['row2']) & \
#             (merged_df['col1'] == merged_df['col2']) & \
#             (merged_df['row1'] == merged_df['row3']+1) & \
#             (merged_df['col1'] == merged_df['col3'])

### 2025 Mar DESY TB - main telescope from Run46
# condition = (merged_df['row1'] == merged_df['row0']) & \
#             (merged_df['col1'] == merged_df['col0']) & \
#             (merged_df['row1'] == merged_df['row2']) & \
#             (merged_df['col1'] == merged_df['col2']) & \
#             (merged_df['row1'] == merged_df['row3']) & \
#             (merged_df['col1'] == merged_df['col3'])

### 2025 Mar DESY TB - second telescope upto Run45
# condition = (merged_df['row2'] == merged_df['row3']) & \
            # (merged_df['col2'] == merged_df['col3']) & \
            # (merged_df['row2'] == merged_df['row0']+1) & \
            # (merged_df['col2'] == merged_df['col0']+1) & \
            # (merged_df['row2'] == merged_df['row1']) & \
            # (merged_df['col2'] == merged_df['col1'])

### 2025 Mar DESY TB - second telescope from Run46
condition = (merged_df['row1'] == merged_df['row0']) & \
            (merged_df['col1'] == merged_df['col0']) & \
            (merged_df['row1'] == merged_df['row2']) & \
            (merged_df['col1'] == merged_df['col2']) & \
            (merged_df['row1'] == merged_df['row3']) & \
            (merged_df['col1'] == merged_df['col3'])


selected_data = merged_df[condition]

In [ ]:
# Define a function to check if tuple exists as a row
# def is_tuple_in_df(t, df):
#     return any(row.tolist() == list(t) for _, row in df.iterrows())

In [ ]:
# col_list, row_list = np.meshgrid(np.arange(16),np.arange(16))
# scan_list = list(zip(row_list.flatten(),col_list.flatten()))

# satisfy_df = merged_df[condition].sort_values(by=['nevt'], ascending=False).reset_index(drop=True)
# non_satisfy_df = merged_df[~condition].sort_values(by=['nevt'], ascending=False).reset_index(drop=True)

# # Filter elements not present
# missing_pixels_board0 = [item for item in scan_list if not is_tuple_in_df(item, satisfy_df[[f'row{board_to_analyze[0]}', f'col{board_to_analyze[0]}']])]
# missing_pixels_board1 = [item for item in scan_list if not is_tuple_in_df(item, satisfy_df[[f'row{board_to_analyze[1]}', f'col{board_to_analyze[1]}']])]
# missing_pixels_board2 = [item for item in scan_list if not is_tuple_in_df(item, satisfy_df[[f'row{board_to_analyze[2]}', f'col{board_to_analyze[2]}']])]

# missing_pix_dict = {
#     board_to_analyze[0]: defaultdict(list),
#     board_to_analyze[1]: defaultdict(list),
#     board_to_analyze[2]: defaultdict(list),
# }

# for board_num, missing_pixels in zip(board_to_analyze, [missing_pixels_board0, missing_pixels_board1, missing_pixels_board2]):
#     for missing_pixel in missing_pixels:
#         board_key = f'board{board_num}'
#         row_key = f'row{board_num}'
#         col_key = f'col{board_num}'
#         res_key = f'res{board_num}'
#         err_key = f'err{board_num}'

#         tmp_df = non_satisfy_df.loc[(non_satisfy_df[row_key] == missing_pixel[0]) & (non_satisfy_df[col_key] == missing_pixel[1])]
#         if not tmp_df.empty:
#             missing_pix_dict[board_num]['row'].append(tmp_df.iloc[0][row_key])
#             missing_pix_dict[board_num]['col'].append(tmp_df.iloc[0][col_key])
#             missing_pix_dict[board_num]['res'].append(tmp_df.iloc[0][res_key])
#             missing_pix_dict[board_num]['err'].append(tmp_df.iloc[0][err_key])

# id_want_drop = list(set([0,1,2,3]) - set(board_to_analyze))[0]
# selected_data = merged_df[condition]
# selected_data = selected_data.drop(columns=[f'row{id_want_drop}', f'col{id_want_drop}', 'nevt'])
# selected_data.reset_index(drop=True, inplace=True)

In [ ]:
# selected_data[['row0', 'col0', 'res0', 'err0', 'nevt']].sort_values(by='res0').head(10)
# selected_data.loc[(selected_data['row0'] == 5) & (selected_data['col0'] == 6)]

In [ ]:
# from pathlib import Path

# current_dir = Path('./plot_scripts_for_approval')
# output_mother_dir = current_dir / 'etroc_TB_figs'
# output_mother_dir.mkdir(exist_ok=True)

# ### Now you need change the directory name per campaign
# ### Naming rule is this:
# ### <TB location>_TB_MonthYear
# ### E.g. desy_TB_Apr2024, cern_TB_Sep2023, fnal_TB_Jul2024

# output_campaign_dir = output_mother_dir / 'desy_TB_Apr2024'
# output_campaign_dir.mkdir(exist_ok=True)

In [ ]:
selected_data = selected_data.reset_index(drop=True)
res_columns = [col for col in selected_data.columns if col.startswith('res')]
selected_data[res_columns] = selected_data[res_columns].mask(selected_data[res_columns].abs() > 200, -1)

In [ ]:
plot_resolution_table(input_df=selected_data, board_ids=board_to_draw, tb_loc='desy', fig_config=selected_fig_config,
                      min_resolution=10, max_resolution=100, slides_friendly = False, show_number=True)

In [ ]:
for id in board_to_draw:
    selected_fig_config[id]['title'] = f"{selected_fig_config[id]['name']} \nHV{selected_fig_config[id]['HV']}V OS:{selected_fig_config[id]['offset']}"

plot_resolution_with_pulls(input_df=selected_data, board_ids=board_to_draw, tb_loc='desy',
                           fig_config=selected_fig_config, hist_range=[20, 75], hist_bins=50, print_fit_results=False)